In [1]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

from rdkit.Chem import MCS
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import rdFMCS
from rdkit.Chem import rdMolDescriptors
import timeit

import numpy as np
import pandas as pd
import seaborn as sns


#descriptor
from rdkit.Chem.Descriptors import ExactMolWt
from rdkit.Chem.Crippen import MolLogP
from rdkit.Chem.rdMolDescriptors import CalcNumHBD
from rdkit.Chem.rdMolDescriptors import CalcNumHBA
from rdkit.Chem.rdMolDescriptors import CalcTPSA

import matplotlib.pyplot as plt

/home/ubuntu/anaconda3/envs/tensorflow2_p36/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The rdkit.Chem.MCS module is deprecated; please use rdkit.Chem.rdFMCS instead.
  after removing the cwd from sys.path.


In [2]:
data=pd.read_csv('pubchem_data_formal_charge_length100_20210512.csv')

In [3]:
data

,CID,smiles,MW,LogP,HBA,HBD,TPSA,HBA_HBD,rotational bond,form_charge,length
0,140,CC(CCCC(C)C1CCC2C1(C(CC3C2C(CC4C3(CCC(C4)O)C)O...,436.355260,4.38260,4,4,80.92,8,6,0,53
1,159,CCCCCC(C=CC1C(CC2C1CC(=CCCCC(=O)O)O2)O)O,352.224974,3.40850,4,3,86.99,7,10,0,40
2,246,CC12CCC(CC1CCC3C2CCC4(C3CCC4=O)C)OS(=O)(=O)O,370.181395,3.78620,4,1,80.67,5,2,0,44
3,302,CC(CCC(=O)O)C1CCC2C1(C(CC3C2C(=O)CC4C3(CCC(C4)...,406.271924,3.65690,4,3,94.83,7,4,0,53
4,303,CC(CCC(=O)O)C1CCC2C1(C(CC3C2C(CC4C3(CCC(C4)O)C...,408.287574,3.44870,4,4,97.99,8,4,0,52
5,699,CC12CCC3C(C1CCC2=O)CCC4=C3C=CC(=C4)OS(=O)(=O)O,350.118795,3.29340,4,1,80.67,5,2,0,46
6,734,CC(=CCCC(=CCOP(=O)(O)OP(=O)(O)O)C)C,314.068426,2.90540,4,3,113.29,7,8,0,35
7,755,CC(CCC(=O)NCC(=O)O)C1CCC2C1(C(CC3C2C(CC4C3(CCC...,465.309038,2.56490,5,5,127.09,10,6,0,59
8,1020,CC(=CCCC=CCOP(=O)(O)OP(=O)(O)O)C,300.052776,2.51530,4,3,113.29,7,8,0,32
9,1039,CC1C2CC3C(C(C(N3)CC4=C(C(C(=N4)CC5C(C(C(N5)CC(...,580.398856,4.78490,6,5,123.05,11,8,0,82


In [4]:
rand_sample=data.sample(n=100000)

In [5]:
rand_sample

,CID,smiles,MW,LogP,HBA,HBD,TPSA,HBA_HBD,rotational bond,form_charge,length
20887582,152377293,CCC(=O)N1CCC(CC1)OC2=CC=C(C=C2)N3C(=S)N(C(=O)C...,428.163060,3.54790,5,0,101.85,5,5,0,64
15282145,120204695,COC1=CC=C(C=C1)C(C(=O)O)NC(=O)CC2=CCCCC2,303.147058,2.82760,3,2,75.63,5,6,0,40
17142997,129355781,CC(C)COC1=NC=C(C=C1)C(=O)NCC2=NN=C3N2CC[C@H](C...,397.172560,2.76260,6,1,81.93,7,6,0,56
16612079,126211891,CC1=CC=C(C=C1)N2C(=O)C(=C(C(=N2)C(=O)O)/C=C/C3...,357.111341,3.28120,5,1,95.98,6,4,0,59
21032988,153386224,C=C(C1=C(SN=C1)C2=CC=CC=N2)NC(CC3=CC=NC=C3)C(=...,379.110296,1.82610,7,2,110.86,9,8,0,54
16608330,126198002,CC(=O)NC1=CC=C(C=C1)NC(=O)COC2=CC=C(C=C2)S(=O)...,453.135842,3.14100,5,3,113.60,8,9,0,63
7788712,56135137,CC(C(=O)NC1=CC(=C(C=C1)SC2=CC=C(C=C2)OC)Cl)NC(...,430.075406,4.84980,5,2,80.57,7,7,0,58
19159876,140601145,CC(C)([C@H](COCC1=CC=CC=C1)C(=O)N2CCCC3(C2)C(C...,477.262757,2.85330,4,2,101.73,6,8,0,70
17514629,133255430,COC1=CC=C(C=C1)C(CC(=O)NC2=CC=C(C=C2)Br)NS(=O)...,426.024890,3.07680,4,2,84.50,6,7,0,51
9402629,68039378,CC1=C(C=C(C=C1)C(=O)NC2=CC=CC(=C2)C(C)(C)N)NC3...,482.254258,4.51812,8,3,121.95,11,6,0,72


In [6]:
data=rand_sample

In [7]:
#filter

data=data[~data.smiles.str.contains("Pb")]
data=data[~data.smiles.str.contains("Cu")]
data=data[~data.smiles.str.contains("Se")]
data=data[~data.smiles.str.contains("Ac")]
data=data[~data.smiles.str.contains("Tc")]
data=data[~data.smiles.str.contains("As")]
data=data[~data.smiles.str.contains("Ni")]

data=data[~data.smiles.str.contains("Eu")]
data=data[~data.smiles.str.contains("Ru")]
data=data[~data.smiles.str.contains("Ti")]
data=data[~data.smiles.str.contains("Li")]
data=data[~data.smiles.str.contains("Ta")]
data=data[~data.smiles.str.contains("Fe")]
data=data[~data.smiles.str.contains("Pt")]
data=data[~data.smiles.str.contains("C-")]
data=data[~data.smiles.str.contains("U")]

data=data[~data.smiles.str.contains("Hg")]
data=data[~data.smiles.str.contains("Zn")]
data=data[~data.smiles.str.contains("Pd")]
data=data[~data.smiles.str.contains("S-")]
data=data[~data.smiles.str.contains("Ga")]
data=data[~data.smiles.str.contains("Ge")]

data=data[~data.smiles.str.contains("V")]
data=data[~data.smiles.str.contains("W")]
data=data[~data.smiles.str.contains("Y")]
data=data[~data.smiles.str.contains("Tl")]
data=data[~data.smiles.str.contains("Rh")]
data=data[~data.smiles.str.contains("Re")]

data=data[~data.smiles.str.contains("Zr")]
data=data[~data.smiles.str.contains("Cd")]
data=data[~data.smiles.str.contains("Rb")]
data=data[~data.smiles.str.contains("Sb")]
data=data[~data.smiles.str.contains("Mn")]
data=data[~data.smiles.str.contains("Mo")]
data=data[~data.smiles.str.contains("Gd")]
data=data[~data.smiles.str.contains("Ag")]


In [8]:
data

,CID,smiles,MW,LogP,HBA,HBD,TPSA,HBA_HBD,rotational bond,form_charge,length
20887582,152377293,CCC(=O)N1CCC(CC1)OC2=CC=C(C=C2)N3C(=S)N(C(=O)C...,428.163060,3.54790,5,0,101.85,5,5,0,64
15282145,120204695,COC1=CC=C(C=C1)C(C(=O)O)NC(=O)CC2=CCCCC2,303.147058,2.82760,3,2,75.63,5,6,0,40
17142997,129355781,CC(C)COC1=NC=C(C=C1)C(=O)NCC2=NN=C3N2CC[C@H](C...,397.172560,2.76260,6,1,81.93,7,6,0,56
16612079,126211891,CC1=CC=C(C=C1)N2C(=O)C(=C(C(=N2)C(=O)O)/C=C/C3...,357.111341,3.28120,5,1,95.98,6,4,0,59
21032988,153386224,C=C(C1=C(SN=C1)C2=CC=CC=N2)NC(CC3=CC=NC=C3)C(=...,379.110296,1.82610,7,2,110.86,9,8,0,54
16608330,126198002,CC(=O)NC1=CC=C(C=C1)NC(=O)COC2=CC=C(C=C2)S(=O)...,453.135842,3.14100,5,3,113.60,8,9,0,63
7788712,56135137,CC(C(=O)NC1=CC(=C(C=C1)SC2=CC=C(C=C2)OC)Cl)NC(...,430.075406,4.84980,5,2,80.57,7,7,0,58
19159876,140601145,CC(C)([C@H](COCC1=CC=CC=C1)C(=O)N2CCCC3(C2)C(C...,477.262757,2.85330,4,2,101.73,6,8,0,70
17514629,133255430,COC1=CC=C(C=C1)C(CC(=O)NC2=CC=C(C=C2)Br)NS(=O)...,426.024890,3.07680,4,2,84.50,6,7,0,51
9402629,68039378,CC1=C(C=C(C=C1)C(=O)NC2=CC=CC(=C2)C(C)(C)N)NC3...,482.254258,4.51812,8,3,121.95,11,6,0,72


In [9]:
smiles=data['smiles']

In [10]:
len(smiles)

99793

In [ ]:
smiles.to_csv('random_filtered_pubchem20210512.csv',index=False)